# Text Generation with Neural Networks

## Functions for Processing Text

### Reading in files as a string text

In [11]:
def read_file(filepath):
    with open(filepath) as f:
        str_text = f.read()
        
    return str_text

In [12]:
# read_file('moby_dick_four_chapters.txt') 

### Tokenize and Clean Text

In [13]:
import spacy

In [14]:
nlp = spacy.load('en_core_web_md',disable=['parser','ner'])

In [15]:
nlp.max_length = 1198623

In [16]:
def separete_punc(doc_text):
    return [token.text.lower() for token in nlp(doc_text) if token.text not in '\n\n \n\n\n!"-#$%&()--.*+,-/:;<=>?@[\\]^_`{|}~\t\n ']

In [17]:
d = read_file('moby_dick_four_chapters.txt')

In [18]:
tokens = separete_punc(d)

In [20]:
len(tokens)

11338

## Create Sequences of Tokens

In [ ]:
# 25 words --> network predict #26

In [21]:
train_len = 25 + 1

text_sequences = []

for i in range(train_len, len(tokens)):
    seq = tokens[i-train_len:i]
    
    text_sequences.append(seq)

In [23]:
type(text_sequences)

list

In [26]:
' '.join(text_sequences[0])

'call me ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me on'

# Keras

### Keras Tokenization

In [27]:
from keras.preprocessing.text import Tokenizer

In [28]:
tokenizer = Tokenizer()

tokenizer.fit_on_texts(text_sequences)

In [29]:
sequences = tokenizer.texts_to_sequences(text_sequences)

In [33]:
# sequences[0]

In [34]:
# tokenizer.index_word

In [32]:
for i in sequences[0]:
    print(f"{i} : {tokenizer.index_word[i]}")

956 : call
14 : me
263 : ishmael
51 : some
261 : years
408 : ago
87 : never
219 : mind
129 : how
111 : long
954 : precisely
260 : having
50 : little
43 : or
38 : no
314 : money
7 : in
23 : my
546 : purse
3 : and
150 : nothing
259 : particular
6 : to
2713 : interest
14 : me
24 : on


In [36]:
# tokenizer.word_counts

In [39]:
vocabulary_size = len(tokenizer.word_counts)
vocabulary_size

2718

### Convert to Numpy Matrix

In [40]:
import numpy as np

In [41]:
sequences = np.array(sequences)

sequences

array([[ 956,   14,  263, ..., 2713,   14,   24],
       [  14,  263,   51, ...,   14,   24,  957],
       [ 263,   51,  261, ...,   24,  957,    5],
       ...,
       [ 952,   12,  166, ...,  262,   53,    2],
       [  12,  166, 2712, ...,   53,    2, 2718],
       [ 166, 2712,    3, ...,    2, 2718,   26]])

### features labels Split

In [42]:
from keras.utils import to_categorical

In [43]:
X = sequences[:,:-1]

In [44]:
y = sequences[:,-1]

In [46]:
y = to_categorical(y, num_classes=vocabulary_size+1)

In [47]:
seq_len = X.shape[1]

# Creating an LSTM based model

In [51]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding

In [54]:
def create_model(vocabulary_size,seq_len):
    
    model = Sequential()
    
    model.add(Embedding(vocabulary_size,seq_len,input_length=seq_len))
    model.add(LSTM(seq_len*6,return_sequences=True))
    model.add(LSTM(seq_len*6))
    model.add(Dense(seq_len*6,activation='relu'))
    model.add(Dense(vocabulary_size, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    
    model.summary()
    
    return model

### Training the Model

In [55]:
model = create_model(vocabulary_size+1, seq_len)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 25, 25)            67975     
                                                                 
 lstm_2 (LSTM)               (None, 25, 150)           105600    
                                                                 
 lstm_3 (LSTM)               (None, 150)               180600    
                                                                 
 dense_2 (Dense)             (None, 150)               22650     
                                                                 
 dense_3 (Dense)             (None, 2719)              410569    
                                                                 
Total params: 787394 (3.00 MB)
Trainable params: 787394 (3.00 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [57]:
from pickle import dump, load

In [58]:
model.fit(X,y, batch_size=128, epochs=300, verbose=1)

Epoch 1/300
89/89 [==============================] - 8s 74ms/step - loss: 6.8595 - accuracy: 0.0469
Epoch 2/300
89/89 [==============================] - 7s 84ms/step - loss: 6.3840 - accuracy: 0.0529
Epoch 3/300
89/89 [==============================] - 8s 85ms/step - loss: 6.3305 - accuracy: 0.0529
Epoch 4/300
89/89 [==============================] - 8s 88ms/step - loss: 6.1927 - accuracy: 0.0519
Epoch 5/300
89/89 [==============================] - 8s 86ms/step - loss: 6.0875 - accuracy: 0.0540
Epoch 6/300
89/89 [==============================] - 8s 86ms/step - loss: 5.9412 - accuracy: 0.0606
Epoch 7/300
89/89 [==============================] - 7s 83ms/step - loss: 5.8412 - accuracy: 0.0665
Epoch 8/300
89/89 [==============================] - 8s 85ms/step - loss: 5.7650 - accuracy: 0.0682
Epoch 9/300
89/89 [==============================] - 7s 83ms/step - loss: 5.6960 - accuracy: 0.0717
Epoch 10/300
89/89 [==============================] - 8s 85ms/step - loss: 5.6205 - accuracy: 0.0777

89/89 [==============================] - 7s 81ms/step - loss: 3.1094 - accuracy: 0.2594
Epoch 83/300
89/89 [==============================] - 7s 81ms/step - loss: 3.0876 - accuracy: 0.2600
Epoch 84/300
89/89 [==============================] - 7s 84ms/step - loss: 3.0557 - accuracy: 0.2679
Epoch 85/300
89/89 [==============================] - 7s 84ms/step - loss: 3.0388 - accuracy: 0.2725
Epoch 86/300
89/89 [==============================] - 7s 83ms/step - loss: 3.0127 - accuracy: 0.2773
Epoch 87/300
89/89 [==============================] - 7s 82ms/step - loss: 2.9858 - accuracy: 0.2839
Epoch 88/300
89/89 [==============================] - 8s 85ms/step - loss: 2.9731 - accuracy: 0.2857
Epoch 89/300
89/89 [==============================] - 7s 79ms/step - loss: 2.9488 - accuracy: 0.2918
Epoch 90/300
89/89 [==============================] - 7s 84ms/step - loss: 2.9240 - accuracy: 0.2981
Epoch 91/300
89/89 [==============================] - 7s 84ms/step - loss: 2.9227 - accuracy: 0.2898
Epo

89/89 [==============================] - 8s 85ms/step - loss: 1.8689 - accuracy: 0.5196
Epoch 163/300
89/89 [==============================] - 7s 82ms/step - loss: 1.8564 - accuracy: 0.5219
Epoch 164/300
89/89 [==============================] - 7s 84ms/step - loss: 1.8534 - accuracy: 0.5225
Epoch 165/300
89/89 [==============================] - 7s 83ms/step - loss: 1.8372 - accuracy: 0.5282
Epoch 166/300
89/89 [==============================] - 8s 85ms/step - loss: 1.8253 - accuracy: 0.5305
Epoch 167/300
89/89 [==============================] - 8s 86ms/step - loss: 1.8152 - accuracy: 0.5293
Epoch 168/300
89/89 [==============================] - 8s 87ms/step - loss: 1.8104 - accuracy: 0.5295
Epoch 169/300
89/89 [==============================] - 7s 82ms/step - loss: 1.7891 - accuracy: 0.5362
Epoch 170/300
89/89 [==============================] - 7s 83ms/step - loss: 1.7851 - accuracy: 0.5370
Epoch 171/300
89/89 [==============================] - 7s 83ms/step - loss: 1.7710 - accuracy: 0

89/89 [==============================] - 8s 92ms/step - loss: 1.0262 - accuracy: 0.7358
Epoch 243/300
89/89 [==============================] - 8s 91ms/step - loss: 1.0187 - accuracy: 0.7357
Epoch 244/300
89/89 [==============================] - 8s 90ms/step - loss: 0.9965 - accuracy: 0.7375
Epoch 245/300
89/89 [==============================] - 8s 93ms/step - loss: 0.9815 - accuracy: 0.7478
Epoch 246/300
89/89 [==============================] - 8s 89ms/step - loss: 0.9661 - accuracy: 0.7500
Epoch 247/300
89/89 [==============================] - 8s 91ms/step - loss: 0.9539 - accuracy: 0.7557
Epoch 248/300
89/89 [==============================] - 8s 86ms/step - loss: 0.9606 - accuracy: 0.7493
Epoch 249/300
89/89 [==============================] - 8s 88ms/step - loss: 0.9447 - accuracy: 0.7564
Epoch 250/300
89/89 [==============================] - 8s 89ms/step - loss: 0.9422 - accuracy: 0.7566
Epoch 251/300
89/89 [==============================] - 8s 91ms/step - loss: 0.9191 - accuracy: 0

In [60]:
# save the model to file
model.save('mobidick_model.keras')
# save the tokenizer
dump(tokenizer,open('mobidick_model','wb'))

# Generating New Text

In [61]:
from keras.preprocessing.sequence import pad_sequences

In [69]:
def generate_text(model,tokenizer,seq_len,seed_text,num_gen_words):
    
    output_text = []
    
    input_text = seed_text
    
    for i in range(num_gen_words):
        
        encoded_text = tokenizer.texts_to_sequences([input_text])[0]
        
        pad_encoded = pad_sequences([encoded_text],maxlen=seq_len, truncating='pre')
        
        pred_word_ind = model.predict(pad_encoded,verbose=0).argmax(axis=1)[0]
        
        pre_word = tokenizer.index_word[pred_word_ind]
        
        input_text += ' '+ pre_word
        
        output_text.append(pre_word)
    
    
    return ' '.join(output_text)

### Grab a random seed sequence

In [63]:
import random
random_pick = random.randint(0,len(text_sequences))

In [65]:
random_seed_text = text_sequences[random_pick]

In [66]:
seed_text = ' '.join(random_seed_text)
seed_text

"thought i to myself the man 's a human being just as i am he has just as much reason to fear me as i have"

In [70]:
generate_text(model,tokenizer,seq_len,seed_text,num_gen_words=25)

"not be altogether unwarranted but the parent of its funeral where unaccountable abed not ignoring it is inducements about me still now n't touch a"